In [1]:
import os
import time
import random
import datetime
import re
import numpy as np
import pandas as pd
import win32com.client as win32
import pythoncom
import optuna
from optuna.trial import TrialState
import matplotlib.pyplot as plt
import seaborn as sns
import time
# ==============================================================================
# 🔧 [설정] Configuration
# ==============================================================================
class Config:
    FILE_NUMBER = 1
    CURRENT_DIR = os.getcwd()
    
    # 파일 경로 설정
    ASPEN_FILENAME = f"Gambaro_kinetic_fitting_auto_v4_{FILE_NUMBER}.bkp" 
    EXCEL_FILE = os.path.join(CURRENT_DIR, "Target_value.xlsx")
    MANUAL_CSV_PATH = os.path.join(CURRENT_DIR, "manual_iterations_none.csv")
    # HISTORY_FILE_LEGACY = os.path.join(CURRENT_DIR, "merged_optimization_history_sig3.csv") 
    HISTORY_FILE_LEGACY = os.path.join(CURRENT_DIR, "warm_start_v7.csv")     

    OPTUNA_DB_PATH = f"sqlite:///{os.path.join(CURRENT_DIR, 'hdk_opt_v7.db')}"
    STUDY_NAME = "HDK_Kinetics_Optimization_v6" 

    DETAILS_FILE = os.path.join(CURRENT_DIR, "optimization_details_optuna_v7.csv")
    RESULT_FILE = os.path.join(CURRENT_DIR, f"best_params_{ASPEN_FILENAME.replace('.bkp','')}.csv")
    
    MAX_C_NUM = 70

    # Aspen 경로 상수
    PATH_ADS_BASE = r"\Data\Reactions\Reactions\HDK\Input\G_ADS_A\1"
    PATH_RXN_BASE = r"\Data\Reactions\Reactions\HDK\Input"

    # 파라미터 기본값 (Par_16 추가됨)
    BASE_PAR = {
        1: 7.76e1, 2: 2.60e-1, 3: 3.66e1, 4: 8.84e-2,    # Adsorption
        5: 3.60e4, 6: 4.56e-1, 7: 4.74e4, 8: 6.02e-1,    # Ea
        9: 3.63e-8, 10: 5.65e0, 11: 1.79e-8, 12: 5.56e0, # k0
        13: 1.21e2, 14: 1.66e3,                          # Equilibrium
        15: 0.878,                                       # Iso/Normal Ratio (Optimization)
        16: 0.5                                          # [NEW] End/Mid Ratio (Default 0.5)
    }

    # Bounds 설정 (최적화 대상)
    PARAM_BOUNDS = {
        # [Adsorption]
        "Par_1":  (1,    1000.0,   True), # 흡착계수 1차항 파라미터 (변경이력: 0~60 구간 배제 60~6000; 구간 하향 1~1000)
        "Par_2":  (0.15,    0.45,     False), # 흡착계수 tanh항 파라미터 (변경이력: 0.19~0.60; 구간 하향 0.15~0.45)
        "Par_3":  (0.1,     50.0,     True),  # 흡착계수 1차항 파라미터 (변경이력: 상한 축소 3500 -> 50, 이후 유지)
        "Par_4":  (0.05,    0.15,     False), # 흡착계수 tanh항 파라미터 (변경이력: 0.01~0.11; 구간 상향 0.05~0.15)
        
        # [Activation Energy & Pre-exp]
        "Par_5":  (20000.0, 49000.0,  False), # 이성질화반응 활성화에너지 파라미터 (변경이력: 29000~52000; 하한 확대 20000~49000)
        "Par_6":  (0.35,    0.70,     False), # 이성질화반응 활성화에너지 파라미터 (변경이력: 0.27~0.50; 구간 상향 0.35~0.70)
        "Par_7":  (35000.0, 60000.0,  False), # 분해반응 활성화 에너지 파라미터 (변경이력: 25000~49000; 구간 상향 35000~60000)
        "Par_8":  (0.50,    0.80,     False), # 분해반응 활성화 에너지 파라미터 (변경이력: 0.38~0.65; 구간 상향 0.50~0.80)
        "Par_9":  (5e-10,   5e-7,     True),  # 빈도함수 1차항 파라미터 (변경이력: 상한 축소 10^-6 -> 10^-8; 5e-10~4e-8; 상한 확대 5e-7)
        "Par_10": (5.5,     10,       False), # 빈도함수 지수항 파라미터 (변경이력: 5.5~10; 구간 하향: 4.0~8.5)
        "Par_11": (1e-10,   5e-7,     True),  #  빈도함수 1차항 파라미터 (변경이력: 상한 축소 10^-6 -> 10^-8; 1e-11~3e-8; 상한 확대 5e-7)
        "Par_12": (5.3,     10,       False), # 빈도함수 지수항 파라미터 (변경이력: 5.3~10; 구간 하향: 4.0~8.5)
        
        # [Equilibrium]
        "Par_13": (100.0,   180.0,    False), # 평형상수 파라미터 (변경이력: 100~180; 구간 하향: 90~150)
        "Par_14": (800.0,  2000.0,   False), # 평형상수 파라미터 (변경이력: 1000~3000; 구간 하향: 800~2000)

        # [Product Distribution]
        "Par_15": (0.80,    0.95,     False), # Iso/Normal Split (변경이력: 0.85~0.95; 구간 축소 0.85~0.93)
        "Par_16": (0.25,    0.6,     False)  # [NEW] End/Mid Split (0.25 ~ 0.6)
    }

    OPT_KEYS = sorted(BASE_PAR.keys())

# ==============================================================================
# 🛠️ 헬퍼 함수 (계산 로직)
# ==============================================================================
def sig_round(x, digits=3):
    """[핵심] 유효숫자 3자리로 반올림하여 변환"""
    if x == 0: return 0
    return float(f"{x:.{digits}g}")

def log_print(msg):
    timestamp = datetime.datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] {msg}")

def get_comp_name(n, isomer=False):
    if n == 1: return "CH4"
    if n == 2: return "C2H6"
    if n == 3: return "C3H8"
    prefix = "ISO-" if isomer else "N-"
    return f"{prefix}C{n}"

def calc_KL(i, par, isomer=False):
    if not isomer:
        val = 0.1 * par[1] * (np.tanh(i * par[2] - 2) + 1)
    else:
        val = 0.1 * par[3] * (np.tanh(i * par[4] - 2) + 1)
    return val * 1e-6

def calc_Keq(i, par):
    return (par[13] * (i**2 - 9) + par[14] * (i - 3))

def get_kinetics(i, par, type="isom"):
    if type == "isom":
        k0_raw = par[9] * (i ** par[10]) * 1e-4
        E_val = par[5] * (i ** par[6])
    else:
        k0_raw = par[11] * (i ** par[12]) * 1e-4
        E_val = par[7] * (i ** par[8])
    return k0_raw, E_val


# ==============================================================================
# 🏗️ Optimizer Class (검증 로직 추가 버전)
# ==============================================================================
class HDKOptimizer:
    def __init__(self, aspen_filename, excel_path):
        pythoncom.CoInitialize()
        
        aspen_abs_path = os.path.abspath(aspen_filename)
        log_print(f"[Init] Aspen 연결 시도: {aspen_filename}")
        
        if not os.path.exists(aspen_abs_path):
            raise FileNotFoundError(f"Aspen file not found: {aspen_abs_path}")

        self.aspen = win32.DispatchEx('Apwn.Document') 
        self.aspen.InitFromArchive2(aspen_abs_path)
        self.aspen.Visible = True
        self.aspen.SuppressDialogs = 1 
        
        self.target_data = self.load_target_data(excel_path)
        self.cache = {} 
        
        log_print(f"✅ [{aspen_filename}] 준비 완료")

    def load_target_data(self, excel_path):
        try: 
            df = pd.read_excel(excel_path)
            if 'Carbon_Num' in df.columns:
                df = df.rename(columns={'Carbon_Num': 'C'})
            return df
        except Exception as e:
            log_print(f"⚠️ 엑셀 로드 실패: {e}")
            return pd.DataFrame()

    def update_aspen(self, param_dict):
        """Aspen 입력 메인 함수"""
        log_print("  ├─ [Update] 파라미터 입력 및 검증 시작...")
        
        # 1. 파라미터 병합
        current_par = Config.BASE_PAR.copy()
        for k_str, val in param_dict.items():
            key_idx = int(k_str.split('_')[1])
            current_par[key_idx] = val

        # 로그 출력
        param_log = " | ".join([f"P{k}={current_par[k]:.3g}" for k in Config.OPT_KEYS])
        log_print(f"  │  ├─ [Params] {param_log}")
            
        # 2. 각 섹션별 업데이트 (검증 포함)
        self._set_adsorption(current_par)
        self._set_kinetics_and_stoichiometry(current_par)

        # [추가] 파라미터 주입 후 아스펜이 데이터를 처리할 물리적 시간 부여
        log_print("  │  ⏳ 파라미터 입력 완료. 안정화를 위해 5초간 대기합니다...")
        time.sleep(5)
        log_print("  │  └─ 입력 및 검증 완료.")

    def _set_adsorption(self, par):
        """흡착 파라미터 업데이트 및 특정 탄소수(C11, 21, 31) 검증"""
        curr_term = 2
        
        # Normal Paraffins (C1 ~ C70)
        for i in range(1, Config.MAX_C_NUM + 1):
            target_val = np.log(calc_KL(i, par, False))
            
            # [수정] C11, C21, C31만 상세 로그 출력
            is_verbose = (i in [11, 21, 31])
            self._set_and_check_node(
                f"{Config.PATH_ADS_BASE}\\{curr_term}", 
                target_val, 
                f"Ads_Normal_C{i}", 
                verbose=is_verbose
            )
            curr_term += 1
            
        # Isomers (C4 ~ C70)
        for i in range(4, Config.MAX_C_NUM + 1):
            target_val = np.log(calc_KL(i, par, True))
            
            # [수정] C11, C21, C31만 상세 로그 출력
            is_verbose = (i in [11, 21, 31])
            self._set_and_check_node(
                f"{Config.PATH_ADS_BASE}\\{curr_term}", 
                target_val, 
                f"Ads_Isomer_C{i}", 
                verbose=is_verbose
            )
            curr_term += 1

    def _set_kinetics_and_stoichiometry(self, par):
        """반응속도 업데이트(15, 65, 100 체크) 및 스토이키 로깅(7 체크)"""
        rxn_counter = 1
        
        for i in range(5, Config.MAX_C_NUM + 1):
            # [수정] Rxn 15, 65, 100 에 대해서만 속도 파라미터 체크 로그 출력
            is_verbose_rxn = (rxn_counter in [15, 65, 100])

            # --- [Reaction 1] Isomerization ---
            k0_iso, E_iso = get_kinetics(i, par, "isom")
            ln_inv_Keq = -np.log(calc_Keq(i, par))
            
            self._set_and_check_node(f"{Config.PATH_RXN_BASE}\\PRE_EXP\\{rxn_counter}", k0_iso * 1e4, f"Rxn{rxn_counter}_Iso_k0", verbose=is_verbose_rxn)
            self._set_and_check_node(f"{Config.PATH_RXN_BASE}\\ACT_ENERGY\\{rxn_counter}", E_iso * 1e3, f"Rxn{rxn_counter}_Iso_Ea", verbose=is_verbose_rxn)
            self._set_and_check_node(f"{Config.PATH_RXN_BASE}\\DF2_A\\{rxn_counter}", ln_inv_Keq, f"Rxn{rxn_counter}_Iso_Keq")
            
            rxn_counter += 1
            
            # --- [Reaction 2] Cracking ---
            if i >= 6:
                k0_ck, E_ck = get_kinetics(i, par, "ck")
                
                # [수정] Rxn 15, 65, 100 체크
                is_verbose_ck = (rxn_counter in [15, 65, 100])
                
                self._set_and_check_node(f"{Config.PATH_RXN_BASE}\\PRE_EXP\\{rxn_counter}", k0_ck * 1e4, f"Rxn{rxn_counter}_Crack_k0", verbose=is_verbose_ck)
                self._set_and_check_node(f"{Config.PATH_RXN_BASE}\\ACT_ENERGY\\{rxn_counter}", E_ck * 1e3, f"Rxn{rxn_counter}_Crack_Ea", verbose=is_verbose_ck)

                # 스토이키 업데이트 (내부에서 7번 반응 로그 출력)
                self._update_stoichiometry(i, par, rxn_counter)
                
                rxn_counter += 1

    def _update_stoichiometry(self, i, par, rxn_counter):
        target_map = {}
        p15 = par[15]
        p16 = par[16]

        # C6 Cracking은 예외적으로 C3 2분자로 고정
        if i == 6:
            target_map["C3H8"] = 2.0
        else:
            # 1. 중간 물질 범위 설정 (C4 ~ C_{i-4})
            # 예: C8 -> k=4 하나 / C9 -> k=4, 5 두개
            mid_comp_list = list(range(4, i - 3)) 
            mid_count = len(mid_comp_list)
            
            # 2. 전체 가중치 합 계산 (Normalization Factor)
            # 중간물질 개수(각 가중치 1.0) + 양 끝단 2개(각 가중치 P16)
            total_weight = (mid_count * 1.0) + (2.0 * p16)
            
            # 계수 보정 인자 (합계를 2.0으로 만들기 위함)
            scaling = 2.0 / total_weight

            # 3. 각 물질별 계수 할당
            # (1) 중간 물질들: 계수 = 1.0 * scaling
            for k in mid_comp_list:
                coeff = 1.0 * scaling
                target_map[get_comp_name(k, True)] = coeff * p15      # ISO
                target_map[get_comp_name(k, False)] = coeff * (1 - p15) # Normal

            # (2) 양 끝단 (C3 및 C_{i-3}): 계수 = P16 * scaling
            end_coeff = p16 * scaling
            
            # C3H8 (이미 존재할 수 있으므로 더해줌)
            target_map["C3H8"] = target_map.get("C3H8", 0) + end_coeff
            
            # Heavy End (C_{i-3})
            heavy_idx = i - 3
            target_map[get_comp_name(heavy_idx, True)] = target_map.get(get_comp_name(heavy_idx, True), 0) + end_coeff * p15
            target_map[get_comp_name(heavy_idx, False)] = target_map.get(get_comp_name(heavy_idx, False), 0) + end_coeff * (1 - p15)

        # [Log] 결과 확인
        if rxn_counter in [7]:
            actual_sum = sum(target_map.values())
            log_print(f"  📝 [Stoich Log] Rxn {rxn_counter} (Feed C{i}) | P16={p16:.3f} | Scaling={2.0/total_weight:.3f}")
            for comp in sorted(target_map.keys(), key=lambda x: (len(x), x)):
                if target_map[comp] > 0.0001:
                    print(f"      > {comp}: {target_map[comp]:.4f}")
            print(f"      > Total Coeff Sum: {actual_sum:.4f}")


        # 2. Aspen 노드 업데이트 (순차 덮어쓰기)
        rxn_node_path = f"{Config.PATH_RXN_BASE}\\COEF1\\{rxn_counter}"
        try:
            rxn_node = self.aspen.Tree.FindNode(rxn_node_path)
            if not rxn_node: return
            
            unique_comp_order = []
            for child in rxn_node.Elements:
                comp_key = child.Name.upper().replace(" MIXED", "").strip()
                if comp_key not in unique_comp_order: unique_comp_order.append(comp_key)

            all_nodes = [child for child in rxn_node.Elements]
            for idx, comp_key in enumerate(unique_comp_order):
                if idx >= len(all_nodes): break
                val = 0.0
                for k, v in target_map.items():
                    if k.upper() == comp_key:
                        val = v
                        break
                all_nodes[idx].Value = val
        except: pass

    def _set_and_check_node(self, path, value, context="", verbose=False):
        """
        [핵심] 값을 입력하고 즉시 읽어서 검증하는 함수
        node.Value = value 입력 후, check = node.Value로 확인
        """
        try:
            node = self.aspen.Tree.FindNode(path)
            if node:
                # 1. 값 입력
                node.Value = value
                
                # 2. 값 확인 (Read-back)
                check_val = node.Value
                
                # 3. 비교 (오차 허용 범위 설정)
                # Aspen은 float 저장 시 미세한 오차가 있을 수 있음
                diff = abs(check_val - value)
                is_match = diff < 1e-5 or (abs(value) > 1e-9 and diff/abs(value) < 0.001)

                if not is_match:
                    log_print(f"      🚨 [MISMATCH] {context}: Input({value:.4g}) != Read({check_val:.4g})")
                elif verbose:
                    # 상세 확인 요청 시 로그 출력 (정상인 경우)
                    log_print(f"      🆗 [Check] {context}: {value:.4g} == {check_val:.4g}")
            else:
                # 노드를 못 찾은 경우
                log_print(f"      ⚠️ [MISSING] Node Not Found: {context}")
                # log_print(f"          Path: {path}") # 필요시 경로 출력

        except Exception as e:
            if context: log_print(f"      ⚠️ Err {context}: {e}")

    # [1] 타임아웃 기능이 강화된 시뮬레이션 실행 함수
    def run_simulation(self, timeout=2000):
        """비동기 방식으로 Aspen을 실행하고 2000초가 지나면 강제 종료"""
        log_print(f"  ├─ [Sim] 비동기 엔진 실행 (제한시간: {timeout}초)")
        start_time = time.time()
        
        # Run2(1)는 비동기 실행을 의미합니다.
        self.aspen.Engine.Run2(1)
        
        while self.aspen.Engine.IsRunning:
            elapsed = time.time() - start_time
            if elapsed > timeout:
                log_print(f"  🚨 [Timeout] {elapsed:.1f}초 경과. 계산 강제 중단!")
                self.aspen.Engine.Stop() 

                # [중요] 중단 명령 후 아스펜이 프로세스를 완전히 정리할 시간 필요
                log_print("  │  ⏳ 엔진 정리를 위해 15초간 대기합니다...")
                time.sleep(15) 
                
                return False # 타임아웃 발생 시 False 반환
            
            time.sleep(10) # 10초마다 상태 체크 (부하 감소)
            
        final_elapsed = time.time() - start_time
        log_print(f"  │  └─ 시뮬레이션 완료 (소요시간: {final_elapsed:.2f}초)")
        
        # 정상 종료 후에도 다음 입력을 위해 짧은 휴식
        time.sleep(2)
        return True # 정상 종료 시 True 반환
        
    def get_results(self):
        # (기존 코드 유지)
        return super(HDKOptimizer, self).get_results() if hasattr(super(HDKOptimizer, self), 'get_results') else self._original_get_results()

    # 편의를 위해 기존 get_results 코드 복사 (상속 구조가 아니므로)
    def _original_get_results(self):
        log_print("  ├─ [Result] 결과 데이터 수집 시작...")
        stream_map = {
            "P-T343": "T343", "P-T351": "T351", "P-T359": "T359",
            "P-P415": "P415", "P-WHSV1": "WHSV1"
        }
        results = []
        for s_idx, (s_name, c_name) in enumerate(stream_map.items()):
            base_path = f"\\Data\\Streams\\{s_name}\\Output\\MASSFLOW\\MIXED"
            try:
                if self.aspen.Tree.FindNode(base_path) is None: continue
            except: continue
            stream_data = self._extract_stream_data(base_path, c_name)
            results.extend(stream_data)
        return pd.DataFrame(results)

    def _extract_stream_data(self, base_path, case_name):
        buf = []
        total = 0.0
        for i in range(1, Config.MAX_C_NUM + 1):
            n_node_name = f"N-C{i}" if i > 3 else (f"C{i}H{2*i+2}" if i <= 3 else "")
            val_n = self._get_node_value(f"{base_path}\\{n_node_name}")
            buf.append({'Case': case_name, 'C': i, 'Type': 'Normal', 'Flow': val_n})
            total += val_n
            val_i = 0.0
            if i > 3: val_i = self._get_node_value(f"{base_path}\\ISO-C{i}")
            buf.append({'Case': case_name, 'C': i, 'Type': 'Isomer', 'Flow': val_i})
            total += val_i
        if total > 0:
            for b in buf: b['Sim_Value'] = b['Flow'] / total * 100
        else:
            for b in buf: b['Sim_Value'] = 0.0
        return buf

    def _get_node_value(self, path):
        try:
            node = self.aspen.Tree.FindNode(path)
            return node.Value if node else 0.0
        except: return 0.0
    
    # save_iteration_details 메서드는 기존 유지
    def save_iteration_details(self, merged_df, trial_number):
        try:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            merged_df['Source_File'] = Config.ASPEN_FILENAME
            merged_df['Timestamp'] = timestamp
            merged_df['Trial'] = trial_number 
            merged_df['SSE_Contrib'] = (merged_df['Exp_Value'] - merged_df['Sim_Value']) ** 2
            
            save_cols = ['Source_File', 'Timestamp', 'Trial', 'Case', 'C', 'Type', 'Exp_Value', 'Sim_Value', 'SSE_Contrib']
            df_save = merged_df[save_cols]
            
            hdr = not os.path.exists(Config.DETAILS_FILE)
            df_save.to_csv(Config.DETAILS_FILE, mode='a', header=hdr, index=False, float_format='%.3e')
        except Exception as e:
            log_print(f"⚠️ 상세 결과 저장 실패: {e}")


    def plot_hc_distribution(self, merged_df, trial_number):
        import matplotlib.pyplot as plt
        from mpl_toolkits.axes_grid1.inset_locator import inset_axes

        plt.rcParams['font.family'] = 'Arial'
        fig, ax = plt.subplots(figsize=(11, 7))
        
        academic_colors = ['#000000', '#FF0000', '#0000FF', '#008000', '#FF8C00', '#8B008B']
        cases = sorted(merged_df['Case'].unique())
        color_map = {case: academic_colors[i % len(academic_colors)] for i, case in enumerate(cases)}

        def draw_contents(target_ax, is_inset=False):
            for case in cases:
                case_data = merged_df[merged_df['Case'] == case]
                color = color_map[case]
                for c_type, marker, ls in [('Normal', '^', '-'), ('Isomer', 's', '--')]:
                    subset = case_data[case_data['Type'] == c_type].sort_values('C')
                    if not subset.empty:
                        # 실험값 scatter
                        target_ax.scatter(subset['C'], subset['Exp_Value'], edgecolor=color, facecolor='none', 
                                         marker=marker, s=40 if not is_inset else 20, linewidth=0.8,
                                         label=f"{case} {c_type[:1]} (Exp)" if not is_inset else "")
                        
                        # 시뮬레이션값 line (label 추가하여 범례에 표시)
                        target_ax.plot(subset['C'], subset['Sim_Value'], color=color, 
                                      linestyle=ls, linewidth=1.5 if not is_inset else 1.0,
                                      label=f"{case} {c_type[:1]} (Sim)" if not is_inset else "")

        # 메인 및 확대도 그리기
        draw_contents(ax)
        
        # [수정] y축 범위 설정: 데이터 최대값의 1.3배로 설정하여 상단 범례 공간 확보
        max_val = max(merged_df['Exp_Value'].max(), merged_df['Sim_Value'].max())
        ax.set_ylim(-0.1, max_val * 1.3) 

        ax.set_title(f"HC distribution in iter={trial_number}", fontsize=14)
        ax.set_xlim(0, 72); ax.set_xticks(range(1, 71, 5)); ax.grid(True, linestyle=':', alpha=0.4)
        ax.set_xlabel("Carbon number"); ax.set_ylabel("Mass fraction [%]")

        # 확대도 (C50-C70 구간)
        ax_ins = inset_axes(ax, width="40%", height="35%", loc='lower right', 
                            bbox_to_anchor=(-0.05, 0.1, 1, 1), bbox_transform=ax.transAxes)
        draw_contents(ax_ins, is_inset=True)
        ax_ins.set_xlim(50, 70); ax_ins.set_ylim(-0.05, 0.5); ax_ins.tick_params(labelsize=8); ax_ins.grid(True, linestyle=':', alpha=0.3)

        # [수정] 범례: 폰트 크기를 조절하고 3열(ncol=3)로 배치하여 높이 차지 최소화
        ax.legend(loc='upper right', fontsize=7, ncol=3, frameon=True, edgecolor='black')
        
        plot_dir = os.path.join(Config.CURRENT_DIR, "plots")
        if not os.path.exists(plot_dir): os.makedirs(plot_dir)
        plt.savefig(os.path.join(plot_dir, f"trial_{trial_number}.png"), dpi=300)
        plt.close()

# ==============================================================================
# 🎯 Optuna Objective
# ==============================================================================
# [3] 타임아웃을 감지하여 안전하게 실행하는 Objective 함수
def objective(trial, optimizer):
    trial_num = trial.number
    param_dict = {k: sig_round(trial.suggest_float(k, *v[:2], log=v[2]), 3) for k, v in Config.PARAM_BOUNDS.items()}
    
    try:
        optimizer.update_aspen(param_dict)
        
        # 시뮬레이션 실행 및 성공 여부 확인
        if not optimizer.run_simulation(timeout=2000):
            log_print(f"  ❌ Trial #{trial_num} 타임아웃 발생 -> Penalty 1e9 부여")
            return 1e9 # 그래프를 그리지 않고 즉시 종료
        
        df_sim = optimizer.get_results()
        merged = pd.merge(optimizer.target_data, df_sim, on=['Case', 'C', 'Type'], how='inner')
        
        if merged.empty: return 1e9
        
        sse = ((merged['Exp_Value'] - merged['Sim_Value']) ** 2).sum()
        
        # 성공한 경우에만 그래프 생성 및 상세 저장
        optimizer.plot_hc_distribution(merged, trial_num)
        optimizer.save_iteration_details(merged, trial_num)
        
        return sse

    except Exception as e:
        log_print(f"⚠️ Error in Trial #{trial_num}: {e}")
        return 1e9

# ==============================================================================
# 📥 데이터 마이그레이션
# ==============================================================================
def preload_existing_data(study, optimizer):
    """과거 데이터를 로드하여 Optuna Study에 주입 (Par15/16 기본값 처리 포함)"""
    if len(study.trials) > 0:
        log_print("ℹ️ 이미 DB에 데이터가 존재하므로 CSV 로드를 건너뜁니다.")
        return

    log_print("📂 [Warm Start] 과거 CSV 데이터 로드 및 마이그레이션 중...")
    
    files_to_load = [f for f in [Config.MANUAL_CSV_PATH, Config.HISTORY_FILE_LEGACY] if os.path.exists(f)]
    count = 0
    
    DEFAULT_PAR15 = 0.878 
    DEFAULT_PAR16 = 0.5   

    for fpath in files_to_load:
        try:
            df = pd.read_csv(fpath)
            # 필요한 컬럼만 추출하여 반복
            # (대용량일 경우 iterrows는 느리지만, 로직 복잡성 때문에 유지)
            for _, row in df.iterrows():
                try:
                    params_dist = {}
                    
                    # 1. 파라미터 값 추출 및 정제
                    for k in range(1, 17): 
                        col_name = f"Par_{k}"
                        val = None
                        
                        if col_name in row:
                            val = row[col_name]
                        elif k == 15: # Par_15가 없으면 기본값
                            val = DEFAULT_PAR15
                        elif k == 16: # Par_16이 없으면 기본값
                            val = DEFAULT_PAR16
                        
                        if val is None or pd.isna(val):
                            raise ValueError("Missing param")
                            
                        params_dist[col_name] = sig_round(float(val), 3)

                    # 2. SSE 값 추출
                    sse = row.get('SSE', row.get('Value', None))
                    if sse is None or pd.isna(sse): continue
                    
                    # 3. 캐시 등록
                    param_values = [params_dist[f"Par_{k}"] for k in sorted(Config.BASE_PAR.keys())]
                    optimizer.cache[tuple(param_values)] = float(sse)
                    
                    # 4. Optuna Trial 주입
                    trial = optuna.trial.create_trial(
                        params=params_dist, 
                        distributions={k: optuna.distributions.FloatDistribution(*v) for k, v in Config.PARAM_BOUNDS.items()},
                        value=float(sse),
                        state=TrialState.COMPLETE
                    )
                    study.add_trial(trial)
                    count += 1
                    
                except Exception: continue

        except Exception as e:
            log_print(f"⚠️ {os.path.basename(fpath)} 로드 중 에러: {e}")
            
    log_print(f"✅ 총 {count}개의 과거 데이터가 로드되었습니다.")

# ==============================================================================
# 🚀 메인 실행 블록
# ==============================================================================
if __name__ == "__main__":
    log_print(f"🚀 최적화 시작 (Optuna TPE + SigFig 3): {Config.ASPEN_FILENAME}")
    
    opt_handler = HDKOptimizer(Config.ASPEN_FILENAME, Config.EXCEL_FILE)
    
    study = optuna.create_study(
        study_name=Config.STUDY_NAME,
        storage=Config.OPTUNA_DB_PATH,
        load_if_exists=True,
        direction="minimize",
        sampler=optuna.samplers.TPESampler(multivariate=True, seed=Config.FILE_NUMBER) 
    )
    
    # Warm Start (과거 이력 로드)
    preload_existing_data(study, opt_handler)
    
    # 최적화 수행
    study.optimize(lambda trial: objective(trial, opt_handler), n_trials=40, catch=(Exception,))

    log_print(f"\n🏆 [Current Kernel] 최적화 종료")
    log_print(f"Best SSE: {study.best_value}")
    log_print(f"Best Params: {study.best_params}")
    
    best_df = pd.DataFrame([study.best_params])
    best_df['SSE'] = study.best_value
    best_df.to_csv(Config.RESULT_FILE, index=False)

[13:14:24] 🚀 최적화 시작 (Optuna TPE + SigFig 3): Gambaro_kinetic_fitting_auto_v4_1.bkp
[13:14:24] [Init] Aspen 연결 시도: Gambaro_kinetic_fitting_auto_v4_1.bkp
[13:15:29] ✅ [Gambaro_kinetic_fitting_auto_v4_1.bkp] 준비 완료


C:\Users\ryu4\anaconda3\envs\your_env\Lib\site-packages\optuna\_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2026-01-29 13:15:30,482] A new study created in RDB with name: HDK_Kinetics_Optimization_v6


[13:15:30] 📂 [Warm Start] 과거 CSV 데이터 로드 및 마이그레이션 중...
[13:15:37] ✅ 총 348개의 과거 데이터가 로드되었습니다.
[13:15:37]   ├─ [Update] 파라미터 입력 및 검증 시작...
[13:15:37]   │  ├─ [Params] P1=163 | P2=0.22 | P3=26.1 | P4=0.0762 | P5=2.67e+04 | P6=0.517 | P7=4.38e+04 | P8=0.548 | P9=6.52e-08 | P10=6.07 | P11=2.31e-07 | P12=6.27 | P13=140 | P14=1.73e+03 | P15=0.887 | P16=0.583
[13:15:37]       🆗 [Check] Ads_Normal_C11: -10.69 == -10.69
[13:15:37]       🆗 [Check] Ads_Normal_C21: -10.34 == -10.34
[13:15:37]       🆗 [Check] Ads_Normal_C31: -10.33 == -10.33
[13:15:38]       🆗 [Check] Ads_Isomer_C11: -14.58 == -14.58
[13:15:38]       🆗 [Check] Ads_Isomer_C21: -13.33 == -13.33
[13:15:38]       🆗 [Check] Ads_Isomer_C31: -12.56 == -12.56
[13:15:38]   📝 [Stoich Log] Rxn 7 (Feed C8) | P16=0.583 | Scaling=0.923
      > C3H8: 0.5383
      > N-C4: 0.1043
      > N-C5: 0.0608
      > ISO-C4: 0.8190
      > ISO-C5: 0.4775
      > Total Coeff Sum: 2.0000
[13:15:39]       🆗 [Check] Rxn15_Crack_k0: 1.349 == 1.349
[13:15:39]      

[I 2026-01-29 14:08:59,686] Trial 348 finished with value: 1000000000.0 and parameters: {'Par_1': 162.77472775917917, 'Par_2': 0.22029326652350034, 'Par_3': 26.131880413847544, 'Par_4': 0.07621584586867376, 'Par_5': 26713.710801581827, 'Par_6': 0.517280106460528, 'Par_7': 43759.33054730111, 'Par_8': 0.5475934551835688, 'Par_9': 6.524818283279799e-08, 'Par_10': 6.070630401126767, 'Par_11': 2.306453797407928e-07, 'Par_12': 6.268419215171448, 'Par_13': 140.1828695262291, 'Par_14': 1726.1798529170155, 'Par_15': 0.8870410620609717, 'Par_16': 0.5830601328907221}. Best is trial 17 with value: 47.42631401.


[14:08:59]   ❌ Trial #348 타임아웃 발생 -> Penalty 1e9 부여
[14:08:59]   ├─ [Update] 파라미터 입력 및 검증 시작...
[14:08:59]   │  ├─ [Params] P1=144 | P2=0.253 | P3=0.528 | P4=0.111 | P5=3.47e+04 | P6=0.489 | P7=4.05e+04 | P8=0.594 | P9=4.25e-08 | P10=5.83 | P11=8.23e-10 | P12=6.19 | P13=139 | P14=1.42e+03 | P15=0.869 | P16=0.315
[14:08:59]       🆗 [Check] Ads_Normal_C11: -10.64 == -10.64
[14:09:00]       🆗 [Check] Ads_Normal_C21: -10.46 == -10.46
[14:09:00]       🆗 [Check] Ads_Normal_C31: -10.46 == -10.46
[14:09:00]       🆗 [Check] Ads_Isomer_C11: -17.81 == -17.81
[14:09:00]       🆗 [Check] Ads_Isomer_C21: -16.48 == -16.48
[14:09:00]       🆗 [Check] Ads_Isomer_C31: -16.12 == -16.12
[14:09:01]   📝 [Stoich Log] Rxn 7 (Feed C8) | P16=0.315 | Scaling=1.227
      > C3H8: 0.3865
      > N-C4: 0.1607
      > N-C5: 0.0506
      > ISO-C4: 1.0663
      > ISO-C5: 0.3359
      > Total Coeff Sum: 2.0000
[14:09:01]       🆗 [Check] Rxn15_Crack_k0: 0.00394 == 0.00394
[14:09:01]       🆗 [Check] Rxn15_Crack_Ea: 1.772e+0

...

[06:21:30]   ├─ [Update] 파라미터 입력 및 검증 시작...
[06:21:30]   │  ├─ [Params] P1=189 | P2=0.33 | P3=38.9 | P4=0.0981 | P5=3.21e+04 | P6=0.431 | P7=3.59e+04 | P8=0.534 | P9=2.53e-08 | P10=5.98 | P11=2.98e-07 | P12=5.87 | P13=135 | P14=1.24e+03 | P15=0.926 | P16=0.496
[06:21:30]       🆗 [Check] Ads_Normal_C11: -10.22 == -10.22
[06:21:30]       🆗 [Check] Ads_Normal_C21: -10.18 == -10.18
[06:21:30]       🆗 [Check] Ads_Normal_C31: -10.18 == -10.18
[06:21:30]       🆗 [Check] Ads_Isomer_C11: -13.75 == -13.75
[06:21:31]       🆗 [Check] Ads_Isomer_C21: -12.4 == -12.4
[06:21:31]       🆗 [Check] Ads_Isomer_C31: -11.88 == -11.88
[06:21:31]   📝 [Stoich Log] Rxn 7 (Feed C8) | P16=0.496 | Scaling=1.004
      > C3H8: 0.4980
      > N-C4: 0.0743
      > N-C5: 0.0369
      > ISO-C4: 0.9297
      > ISO-C5: 0.4611
      > Total Coeff Sum: 2.0000
[06:21:32]       🆗 [Check] Rxn15_Crack_k0: 0.6442 == 0.6442
[06:21:32]       🆗 [Check] Rxn15_Crack_Ea: 1.353e+08 == 1.353e+08
[06:22:58]       🆗 [Check] Rxn65_Crack_k0:

[I 2026-01-30 07:08:01,267] Trial 575 finished with value: 1000000000.0 and parameters: {'Par_1': 188.52361757957465, 'Par_2': 0.3301901814327527, 'Par_3': 38.932491580948515, 'Par_4': 0.09814175543790737, 'Par_5': 32055.187703741776, 'Par_6': 0.43115214330105833, 'Par_7': 35896.17192124244, 'Par_8': 0.5340001722792493, 'Par_9': 2.531870891128175e-08, 'Par_10': 5.979786185937686, 'Par_11': 2.9786482733511237e-07, 'Par_12': 5.8701221559040535, 'Par_13': 134.59283818085513, 'Par_14': 1244.5433586041343, 'Par_15': 0.9257607805329584, 'Par_16': 0.495653007174007}. Best is trial 17 with value: 47.42631401.


[07:08:01]   ❌ Trial #575 타임아웃 발생 -> Penalty 1e9 부여
[07:08:01]   ├─ [Update] 파라미터 입력 및 검증 시작...
[07:08:01]   │  ├─ [Params] P1=63.5 | P2=0.161 | P3=1.34 | P4=0.0619 | P5=3.63e+04 | P6=0.473 | P7=3.92e+04 | P8=0.636 | P9=6.42e-08 | P10=5.84 | P11=1.15e-10 | P12=7.54 | P13=160 | P14=1.64e+03 | P15=0.865 | P16=0.298
[07:08:01]       🆗 [Check] Ads_Normal_C11: -12.22 == -12.22
[07:08:01]       🆗 [Check] Ads_Normal_C21: -11.34 == -11.34
[07:08:01]       🆗 [Check] Ads_Normal_C31: -11.28 == -11.28
[07:08:02]       🆗 [Check] Ads_Isomer_C11: -17.84 == -17.84
[07:08:02]       🆗 [Check] Ads_Isomer_C21: -16.75 == -16.75
[07:08:02]       🆗 [Check] Ads_Isomer_C31: -15.91 == -15.91
[07:08:03]   📝 [Stoich Log] Rxn 7 (Feed C8) | P16=0.298 | Scaling=1.253
      > C3H8: 0.3734
      > N-C4: 0.1692
      > N-C5: 0.0504
      > ISO-C4: 1.0840
      > ISO-C5: 0.3230
      > Total Coeff Sum: 2.0000
[07:08:04]       🆗 [Check] Rxn15_Crack_k0: 0.01577 == 0.01577
[07:08:04]       🆗 [Check] Rxn15_Crack_Ea: 1.904e+

[I 2026-01-30 07:55:58,340] Trial 588 finished with value: 1000000000.0 and parameters: {'Par_1': 63.5184326307978, 'Par_2': 0.16099707554803258, 'Par_3': 1.3416265266089888, 'Par_4': 0.06186343940523946, 'Par_5': 36291.102891955044, 'Par_6': 0.4725474736767619, 'Par_7': 39244.06610515542, 'Par_8': 0.6358874577801882, 'Par_9': 6.416214536852715e-08, 'Par_10': 5.839923717478646, 'Par_11': 1.15218128593231e-10, 'Par_12': 7.535130718036241, 'Par_13': 159.5928705837359, 'Par_14': 1635.7461888475189, 'Par_15': 0.8654276720375339, 'Par_16': 0.2975035124238572}. Best is trial 17 with value: 47.42631401.


[07:55:58]   ❌ Trial #588 타임아웃 발생 -> Penalty 1e9 부여
[07:55:58] 
🏆 [Current Kernel] 최적화 종료
[07:55:58] Best SSE: 47.42631401
[07:55:58] Best Params: {'Par_1': 205.0, 'Par_2': 0.216, 'Par_3': 7.59, 'Par_4': 0.0634, 'Par_5': 36000.0, 'Par_6': 0.434, 'Par_7': 45900.0, 'Par_8': 0.601, 'Par_9': 1.75e-08, 'Par_10': 6.14, 'Par_11': 3.49e-10, 'Par_12': 6.92, 'Par_13': 133.0, 'Par_14': 1480.0, 'Par_15': 0.893, 'Par_16': 0.367}
